In [1]:
# Import libraries
import pandas as pd
import numpy as np
import pickle
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Read in dataset
df=pd.read_csv('Data/90contaminants_RadiumRevised.csv')

In [3]:
df.head()

,utilityid,contaminant_name,date,labid,value,year,state
0,NH2512010,Diquat,2010-03-24,103-407-1,ND,2010,NH
1,NH2512010,Endothall,2010-03-24,103-407-1,ND,2010,NH
2,NH2512010,Endrin,2010-03-24,103-407-1,ND,2010,NH
3,NH2512010,Endrin aldehyde,2010-03-24,103-407-1,ND,2010,NH
4,NH2512010,Ethylbenzene,2010-03-24,8753301,ND,2010,NH


In [4]:
# Remove the asterisk in the value column
df['value'] = df['value'].str.replace('*','')

C:\Users\leowe\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


In [5]:
# Read in the utility characteristics file
fips=pd.read_csv('Data/pwsid2fips.csv')

In [6]:
fips

,pwsid,zipcode2,fips,primarysource,populationservedcount,ownertype,iswholesaler,isoutstandingperformer,issourcewaterprotected,serviceconnectionscount,countyname,state_fips,county_fips,fips_FIMS_MA,pwsname,eparegion,primacyagency,pwstype,primacytype,activitystatus,deactivationdate,zip_code,l,duplicates,new_fips
0,AL0000001,NaN,1001,Ground water,7218.0,Local government,0.0,-,N,2540.0,Autauga,1.0,1.0,1001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,AL0000002,NaN,1001,Ground water,2088.0,Local government,0.0,-,N,562.0,Autauga,1.0,1.0,1001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,AL0000003,NaN,1001,Ground water,1473.0,Local government,0.0,-,N,480.0,Autauga,1.0,1.0,1001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,AL0000013,NaN,1001,Surface water purchased,7320.0,Local government,0.0,-,N,2642.0,Autauga,1.0,1.0,1001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,AL0000017,NaN,1001,Surface water purchased,39603.0,Local government,0.0,-,N,13858.0,Autauga,1.0,1.0,1001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49474,VI1000175,NaN,78030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,St. Thomas Island,78.0,30.0,78030.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
49475,VI1000188,NaN,78030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,St. Thomas Island,78.0,30.0,78030.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
49476,VI1000220,NaN,78030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,St. Thomas Island,78.0,30.0,78030.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
49477,VI1000304,NaN,78030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,St. Thomas Island,78.0,30.0,78030.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [7]:
# Skim through the columns
fips.columns

Index(['pwsid', 'zipcode2', 'fips', 'primarysource', 'populationservedcount',
       'ownertype', 'iswholesaler', 'isoutstandingperformer',
       'issourcewaterprotected', 'serviceconnectionscount', 'countyname',
       'state_fips', 'county_fips', 'fips_FIMS_MA', 'pwsname', 'eparegion',
       'primacyagency', 'pwstype', 'primacytype', 'activitystatus',
       'deactivationdate', 'zip_code', 'l', 'duplicates', 'new_fips'],
      dtype='object')

In [8]:
# Read in the census file
census = pd.read_csv("Data/Census_1982_2018.csv")

In [9]:
census

,fips,year,white,median_household_income,median_year_structure_built,total_pop,housing_density,CPI_annual_2015,deflated_median_household_income,nonwhite
0,1001,1982,0.776045,53953.92,1967.857,32905.33,19.08651,0.407184,53953.92,0.223956
1,1001,1983,0.778883,53959.05,1968.490,33041.04,19.33070,0.420283,53959.05,0.221117
2,1001,1984,0.781411,53967.04,1969.122,33105.06,19.53664,0.438170,53967.04,0.218589
3,1001,1985,0.783684,53979.48,1969.755,33137.38,19.72920,0.453804,53979.48,0.216316
4,1001,1986,0.785757,53997.96,1970.391,33177.94,19.93325,0.462537,53997.96,0.214243
...,...,...,...,...,...,...,...,...,...,...
115943,72153,2014,0.899384,15346.00,1980.000,40391.00,248.46140,0.998814,15346.00,0.100616
115944,72153,2015,0.880200,14708.00,1980.000,39474.00,251.24730,1.000000,14708.00,0.119800
115945,72153,2016,0.841221,14666.00,1980.000,38519.00,254.78690,1.012616,14483.29,0.158779
115946,72153,2017,0.810855,14451.00,1980.000,37585.00,256.14560,1.034185,13973.33,0.189145


In [10]:
# Observe the counts of NaNs for each column
census.isna().sum()

fips                                0
year                                0
white                               0
median_household_income             1
median_year_structure_built         3
total_pop                           0
housing_density                     0
CPI_annual_2015                     0
deflated_median_household_income    1
nonwhite                            0
dtype: int64

In [11]:
# Filter for the non-detects
ND = df[df['value'] == 'ND']

In [12]:
# Filter for the detects
D = df[df['value'] != 'ND']

In [20]:
# Extract the concentration and unit from the value column
D['concentration'] = D['value'].str.replace(',','').str.extract(r'(\S*) \S*').astype(float)
D['units'] = D['value'].str.extract(r'\S* (\S*)')

C:\Users\leowe\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\leowe\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
D['units'].value_counts()

ppb      6053251
ppm      1373216
pCi/L     152638
MFL          584
ppt           11
Name: units, dtype: int64

In [22]:
# Combine the non-detects and detects
dfnew = pd.concat([ND,D])

In [23]:
# Merge with the utility characteristics file
dfnew_fips = dfnew.merge(fips,left_on='utilityid',right_on='pwsid')

In [24]:
# Observe the counts of NaNs for each column
dfnew_fips.isna().sum()

utilityid                         0
contaminant_name                  0
date                              0
labid                       2715179
value                            36
year                              0
state                             0
concentration              20097809
units                      20097809
pwsid                             0
zipcode2                   27050872
fips                              0
primarysource                323877
populationservedcount        323877
ownertype                    430049
iswholesaler                 430049
isoutstandingperformer       430049
issourcewaterprotected       430049
serviceconnectionscount      430049
countyname                   106172
state_fips                   106172
county_fips                  106172
fips_FIMS_MA                 106172
pwsname                    27482656
eparegion                  27482656
primacyagency              27482656
pwstype                    27482656
primacytype                2

In [25]:
# See how many rows are left out after the merge
dfnew_fips.shape

(27588828, 34)

In [26]:
# Drop columns we dont need
dfnew_fips.drop(columns=['pwsname','eparegion','primacyagency','pwstype','activitystatus','deactivationdate','zip_code','l','pwsid','zipcode2'],inplace=True)

In [27]:
# Drop columns we dont need
dfnew_fips.drop(columns=['primacytype','duplicates','new_fips','countyname','state_fips','county_fips','fips_FIMS_MA'],inplace=True)

In [28]:
# Drop columns we dont need
census.drop(columns=['white','median_household_income','CPI_annual_2015'],inplace=True)

In [29]:
# Observe the counts of NaNs for each column
census.isna().sum()

fips                                0
year                                0
median_year_structure_built         3
total_pop                           0
housing_density                     0
deflated_median_household_income    1
nonwhite                            0
dtype: int64

In [30]:
# Merge with the census file
merged=dfnew_fips.merge(census,left_on=['year','fips'],right_on=['year','fips'])

In [31]:
# Observe the counts of NaNs for each column
merged.isna().sum()

utilityid                                  0
contaminant_name                           0
date                                       0
labid                                2708667
value                                     36
year                                       0
state                                      0
concentration                       20083064
units                               20083064
fips                                       0
primarysource                         323877
populationservedcount                 323877
ownertype                             429657
iswholesaler                          429657
isoutstandingperformer                429657
issourcewaterprotected                429657
serviceconnectionscount               429657
median_year_structure_built              590
total_pop                                  0
housing_density                            0
deflated_median_household_income           0
nonwhite                                   0
dtype: int

In [32]:
# Make contaminant names lower case to match the style later
merged['contaminant_name'] = merged['contaminant_name'].str.lower()

In [33]:
# Read in the minimum report limit for each contaminant from an Excel file
DLR=pd.read_excel('Data/MCL&DLR.xlsx',header=None)[[0,2]]
DLR[0] = DLR[0].str.lower()
DLR[0] = DLR[0].str.strip()

In [34]:
# Filter for the non-detects
ND = merged[merged['value'] == 'ND']

In [35]:
# Filter for the detects
D = merged[merged['value'] != 'ND']

In [36]:
# see if the detects and non-detects both contain all contaminants
np.setdiff1d(merged['contaminant_name'].unique(),D['contaminant_name'].unique())

array(['alachlor oxanilic acid'], dtype=object)

In [37]:
# Observe the counts of NaNs for each column
D.isna().sum()

utilityid                                0
contaminant_name                         0
date                                     0
labid                               652727
value                                   36
year                                     0
state                                    0
concentration                           36
units                                   36
fips                                     0
primarysource                        93314
populationservedcount                93314
ownertype                           131153
iswholesaler                        131153
isoutstandingperformer              131153
issourcewaterprotected              131153
serviceconnectionscount             131153
median_year_structure_built            111
total_pop                                0
housing_density                          0
deflated_median_household_income         0
nonwhite                                 0
dtype: int64

In [38]:
# Drop the labid column and all rows with NaNs
train = D.drop(columns=['labid']).dropna()

In [39]:
# Convert all concentration to the ppb unit
ppt = train[train['units'] == 'ppt']
nonppt = train[train['units'] != 'ppt']
ppt['concentration'] = ppt['concentration'] / 1000
ppt['units'] = pd.Series(['ppb']*ppt.shape[0])
ppt=ppt.fillna('ppb')
train=pd.concat([ppt,nonppt])
ppm = train[train['units'] == 'ppm']
nonppm = train[train['units'] != 'ppm']
ppm['concentration'] = ppm['concentration'] * 1000
ppm['units'] = pd.Series(['ppb']*ppm.shape[0])
ppm=ppm.fillna('ppb')

C:\Users\leowe\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\leowe\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\leowe\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [40]:
# Append them back into one dataframe
train=pd.concat([ppm,nonppm])

In [41]:
# Merge with the minimum reporting limit data
train=train.merge(DLR,left_on=['contaminant_name'],right_on=[0])

In [42]:
train.drop(columns=[0],inplace=True)

In [43]:
# Rename column to DLR
train['DLR'] = train[2]
train.drop(columns=[2],inplace=True)

In [44]:
train.head()

,utilityid,contaminant_name,date,value,year,state,concentration,units,fips,primarysource,populationservedcount,ownertype,iswholesaler,isoutstandingperformer,issourcewaterprotected,serviceconnectionscount,median_year_structure_built,total_pop,housing_density,deflated_median_household_income,nonwhite,DLR
0,NH2512010,fluoride,2010-10-05,0.200 ppm,2010,NH,200.0,ppb,33013,Ground water,53.0,Private,0.0,-,-,21.0,1974.75,146445.0,68.02258,67064.8,0.04668,100.0
1,NH0262030,fluoride,2010-07-12,1.30 ppm,2010,NH,1300.0,ppb,33013,Ground water,50.0,Private,0.0,-,-,20.0,1974.75,146445.0,68.02258,67064.8,0.04668,100.0
2,NH0374010,fluoride,2010-09-29,0.200 ppm,2010,NH,200.0,ppb,33013,Ground water,25.0,Private,0.0,-,-,16.0,1974.75,146445.0,68.02258,67064.8,0.04668,100.0
3,NH1181010,fluoride,2010-02-17,1.000 ppm,2010,NH,1000.0,ppb,33013,Surface water purchased,4300.0,Local government,0.0,-,-,1714.0,1974.75,146445.0,68.02258,67064.8,0.04668,100.0
4,NH1181010,fluoride,2010-08-19,1.10 ppm,2010,NH,1100.0,ppb,33013,Surface water purchased,4300.0,Local government,0.0,-,-,1714.0,1974.75,146445.0,68.02258,67064.8,0.04668,100.0


In [45]:
# Filter for samples under the minimum reporting limit to get training data
underDLR = train[(train['concentration'] <= train['DLR']) & (train['concentration'] > 0)]

In [46]:
underDLR.shape

(1013835, 22)

In [47]:
# Convert categorical variables to dummy variables
underDLR=pd.concat([underDLR,pd.get_dummies(underDLR['year'],drop_first=True),
           pd.get_dummies(underDLR['state'],drop_first=True),
           pd.get_dummies(underDLR['primarysource'],drop_first=True),
           pd.get_dummies(underDLR['ownertype'],drop_first=True),
           pd.get_dummies(underDLR['isoutstandingperformer'],drop_first=True),
           pd.get_dummies(underDLR['issourcewaterprotected'],drop_first=True),
           pd.get_dummies(underDLR['contaminant_name'],drop_first=True)], axis=1)

In [48]:
underDLR.head()

,utilityid,contaminant_name,date,value,year,state,concentration,units,fips,primarysource,populationservedcount,ownertype,iswholesaler,isoutstandingperformer,issourcewaterprotected,serviceconnectionscount,median_year_structure_built,total_pop,housing_density,deflated_median_household_income,nonwhite,DLR,2011,2012,2013,2014,2015,2016,2017,AL,AR,AZ,CA,CO,CT,DC,DE,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY,Ground water purchased,Groundwater under influence of surface water,Purchased ground water under influence of surface water source,Surface water,Surface water purchased,Unknown Primary Source,Local government,Native American,Private,Public/Private,State government,N,Y,N,Y,"1,1,2-trichloroethane","1,1-dichloroethylene","1,2,4-trichlorobenzene","1,2-dibromo-3-chloropropane (dbcp)","1,2-dichloroethane","1,2-dichloropropane",17-beta-estradiol,"2,3,7,8-tcdd (dioxin)","2,4,5-tp (silvex)","2,4-d",alachlor (lasso),alachlor esa,alpha-chlordane,alpha-lindane,antimony,arsenic,asbestos,atrazine,barium,benzene,benzo[a]pyrene,beryllium,beta-bhc,bromate,bromodichloromethane,bromoform,cadmium,carbofuran,carbon tetrachloride,chlordane,chlorite,chloroform,chromium (total),"cis-1,2-dichloroethylene",cyanide (free),dalapon,di(2-ethylhexyl) adipate,di(2-ethylhexyl) phthalate,dibromoacetic acid,dibromochloromethane,dichloroacetic acid,dichloromethane (methylene chloride),dinoseb,diquat,endothall,endrin,endrin aldehyde,ethylbenzene,ethylene dibromide,fluoride,glyphosate,heptachlor,heptachlor epoxide,hexachlorobenzene (hcb),hexachlorocyclopentadiene,lindane,mercury (inorganic),methoxychlor,monobromoacetic acid,monochloroacetic acid,monochlorobenzene (chlorobenzene),nitrate,nitrite,o-dichlorobenzene,oxamyl (vydate),p-dichlorobenzene,pentachlorophenol,picloram,polychlorinated biphenyls (pcbs),"radium, combined (-226 & -228)",selenium,simazine,styrene,tetrachloroethylene (perchloroethylene),thallium,toluene,toxaphene,"trans-1,2-dichloroethylene",trichloroacetic acid,trichloroethylene,uranium,vinyl chloride,xylenes (total)
56,NH1861010,fluoride,2012-01-12,0.1000 ppm,2012,NH,100.0,ppb,33013,Ground water,5200.0,Local government,0.0,-,-,2100.0,1976.0,146880.0,68.07397,65303.42,0.049142,100.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
67,NH1193020,fluoride,2012-04-02,0.1000 ppm,2012,NH,100.0,ppb,33013,Ground water,90.0,Private,0.0,-,-,36.0,1976.0,146880.0,68.07397,65303.42,0.049142,100.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
70,NH0851010,fluoride,2012-01-11,0.1000 ppm,2012,NH,100.0,ppb,33013,Ground water,7000.0,Local government,0.0,-,-,2600.0,1976.0,146880.0,68.07397,65303.42,0.049142,100.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
71,NH0851010,fluoride,2012-01-11,0.1000 ppm,2012,NH,100.0,ppb,33013,Ground water,7000.0,Local government,0.0,-,-,2600.0,1976.0,146880.0,68.07397,65303.42,0.049142,100.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [49]:
# Get the target variable and drop columns we dont need for training
target = underDLR['concentration']
underDLR.drop(columns=['utilityid','contaminant_name','date','value','year','state','concentration','units','primarysource','ownertype','isoutstandingperformer','issourcewaterprotected','DLR'],inplace=True)

In [50]:
# Add a dummy varaible fro alachlor oxanilic acid because it is only in test data
underDLR=underDLR.reset_index(drop=True)
underDLR['alachlor oxanilic acid'] = pd.Series([0]*underDLR.shape[0])

In [51]:
# Split into training and validation samples
x_train, x_test, y_train, y_test = train_test_split(underDLR, target)

In [52]:
# Constract a random forest regressor (hyperparameters tuned)
rf = RandomForestRegressor(n_estimators=20,max_depth=80,min_samples_split=10)

In [53]:
# Fit training data
rf.fit(x_train,np.log(y_train))

RandomForestRegressor(max_depth=80, min_samples_split=10, n_estimators=20)

In [54]:
# R-squared value
print(rf.score(x_train,np.log(y_train)))
print(rf.score(x_test,np.log(y_test)))

0.9681761704478581
0.9477118899611117


In [55]:
# Save model
with open("rf.pkl", 'wb') as file:
    pickle.dump(rf, file)